# Table of Content <a id='toc'></a>


&nbsp;&nbsp;&nbsp;&nbsp;[Module 7 - Biopython](#0)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Broad examples of what we can do with Biopython](#1)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Main concept of Biopython](#2)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Objects](#3)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Help - important!](#4)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Accessing Online Databases](#17)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Bio.SeqIO Module](#11)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Reading sequence records from files](#12)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Writing to sequence files](#16)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[`Seq` objects and 'Biological' methods](#10)



&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Now, you can try to solve the exercises.](#18)

&nbsp;&nbsp;&nbsp;&nbsp;[Additional Theory](#19)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[On `SeqRecord` iterators and processing large sequence files with `SeqIO`](#20)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[More on `SeqFeature` objects](#21)


[back to the toc](#toc)

<br>

# Module 7 - Biopython <a id='0'></a>
----------------------------------

<br>

Let's consider a typical task in bioinformatics: we are interested in finding the GC% for some sequences. For this we would need i) open the file, ii) parse and extract the sequence information, and iii) calculate and report their GC%. In a very simple example, we could write something like following: 


In [ ]:
my_seq = {}
identifier = None
with open("data/my_sequences.fa") as input_file:
    for line in input_file:
        if line[0] == ">":
            identifier, *description = line.strip()[1:].split(" ", 1)
            my_seq[identifier] = ""
        else:
            my_seq[identifier] += line.strip().upper()
for identifier in my_seq:
    gc_count = my_seq[identifier].count("G") + my_seq[identifier].count("C")
    gc_pc = gc_count / len(my_seq[identifier]) * 100
    print("GC% for",identifier,"is", gc_pc)

Now let's see how it would look like if we used Biopython:

In [ ]:
from Bio import SeqIO
from Bio.SeqUtils import GC
for rec in SeqIO.parse("data/my_sequences.fa", "fasta"):
    print("GC% for",rec.id,"is",GC(rec.seq))

<br>


[back to the toc](#toc)

<br>

## Broad examples of what we can do with Biopython <a id='1'></a>

- Sequence analysis
  - Motif
  - Search: HMMs, Alignments
  - Restriction
- Structures
  - SCOP
  - PDB
- Database query
- Phylogeny
- Pathway
- And more ...


[back to the toc](#toc)

<br>

## Main concept of Biopython <a id='2'></a>

- I/O interface and parsing abilities for bioinformatics files/DBs
  - Blast
  - FASTA
  - PubMed
  - SwissProt
- Efficient and practical data-structures for bioinformatics data
 - Sequences
 - Alignments
 - Structures
- Methods implementing bioinformatics analysis
 - Translation
 - Classification
 - Phylogeny trees
- Interface to common bioinformatics programs
  - Standalone Blast
  - ClustalW
  - EMBOSS


[back to the toc](#toc)

<br>

## Objects <a id='3'></a>

In Biopython there are many modules and each module contains several major new data types. Objects created with these types serve specific purposes in the above mentioned functionalities. We will focus on sequence analysis; some new objects we will discover are:

- `Seq`
- `SeqRecord`


[back to the toc](#toc)

<br>

## Help - important! <a id='4'></a>

- A relatively detailed tutorial: http://biopython.org/DIST/docs/tutorial/Tutorial.html
- Help on certain concepts and modules: https://biopython.org/wiki/Category%3AWiki_Documentation

In this notebook we will cover basics of the Biopython package:
    - Seq object and alphabets
    - Bio.SeqIO module and SeqRecord object
    - Interacting with external DBs


[back to the toc](#toc)

<br>

## Accessing Online Databases <a id='17'></a>

Biophyton introduces multiple means to interact with commonly used bioinformatics databases over the internet. We will cover the basics of accessing online databases with two databases:

1. NCBI's Entrez
2. UniProt/SwissProt's ExPASy

The Entrez can be queried using the `Bio.Entrez` module. There many different functionalities, but today we will only cover the `.efetch()` method, which we can use to fetch records from Entrez over the internet.

Let's discover this functionality with an example.

<br>

Imagine that you used blast using some sequence as query and got significant hits with the following Entrez sequence ids :

In [ ]:
hits_ids = ['KAJ0778808.1',
             'PWA60940.1',
             'KAI7755456.1',
             'GEU62861.1',
             'AIG92846.1',
             'AIW00681.1',
             'XP_024994640.1',
             'KAI3824653.1',
             'KAD3641182.1',
             'KAI7751279.1']

We would like to gather some information regarding these sequences.

First we will just check the connection and look-up the accessible databases:

In [ ]:
from Bio import Entrez
from Bio import SeqIO
import time 

Entrez.email = "..."  # write you e-mail here. Always tell NCBI who you are
stream = Entrez.einfo()
result = stream.read()
stream.close()

In [ ]:
result

As a result we received some XML  which contains a `<DbList>` which contains a number of `<DbName>`.

The batabase we are interested in is `protein`. 

Let's fetch the first ID:

In [ ]:
%%time
pid = hits_ids[0] # we will fetch the 1st ID in our list
stream = Entrez.efetch(db="protein", id=pid, rettype="gb", retmode="text")

## we receive the data as a text stream in the genbank forma, which we need to interpret:
record = SeqIO.read(stream, "genbank")

stream.close()

What we received is a `SeqRecord` object:

In [ ]:
record

Biopython often stores complex objects using custom types which store their elements in different attributes.

Let's explore this SeqRecord object (you can also browse the [online documentation](https://biopython.org/docs/latest/api/Bio.SeqRecord.html?highlight=seqrecord#module-Bio.SeqRecord))

`SeqRecord` contains many attributes, some of which can be rather complex. We will further examine most commonly used ones:

- seq
- id
- name
- description
- annotations
- features

In [ ]:
print( record.seq )

In [ ]:
record.id , record.description

In [ ]:
record.annotations

In [ ]:
record.features

Now we can download the records for all of our hits, and save the results in genbank files:

In [ ]:
%%time
import os

i = 0
for pid in hits_ids:
    i+=1
    
    print(f"{i}/{len(hits_ids)}")
        
    stream = Entrez.efetch(db="protein", id=pid, rettype="gb", retmode="text")
    record = SeqIO.read(stream, "genbank")
    stream.close()
        
    ## writing the SeqRecord to a genbank file
    SeqIO.write(record , pid + '.gb' , "genbank")
    
    ## wait to be sure to respect ncbi API guidelines
    time.sleep(0.3)
    


[back to the toc](#toc)

<br>

## Bio.SeqIO Module  <a id='11'></a>

The `Bio.SeqIO` module provides a simple and uniform interface to read and parse from and write to various sequence file formats (including multiple sequence alignments). The `Bio.SeqIO` module supports a large number of sequence file formats, including *fasta*, *fastq*, and *genbank* and many more, which can be found [here](https://biopython.org/wiki/SeqIO). All sequences in this method will be accessed as `SeqRecord` objects.


[back to the toc](#toc)

<br>

### Reading sequence records from files <a id='12'></a>

The main function of the module is `.parse()`, which takes a file handle (or filename) and format name, and returns a `SeqRecord` iterator. An iterator is an object that can be iterated upon, meaning that you can traverse through all the values, one by one. In this case `Bio.SeqIO.parse()` method returns an iterator of `SeqRecord` objects extracted and parsed from the input file. We can then iterate over the records and process them in a very efficient manner.

In [ ]:
## reading from a single file

records= []

## SeqIO.parse tries to look for several records in a single file, so we need to put it in a for loop
for rec  in SeqIO.parse( hits_ids[0] + '.gb' , format = 'genbank' ) :
    records.append( rec )

rec

In [ ]:
## reading several files

records = []

for h in hits_ids:
    records += list( SeqIO.parse( h + ".gb", "genbank") ) # using the list() function to go through all records
records

#### micro-exercise

Similarly parse the **example.fa** file and compare the `SeqRecord` objects to the above cell. Which attributes are identical? What are the differences? Why?

We can also create a `SeqRecord` directly. We need to import the class from its module, `Bio.SeqRecord`, first. In order to create its sequence we will also need the `Bio.Seq.Seq` or a similar `Seq` class.

In [ ]:
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

record = SeqRecord(Seq("ACGGCTATCTGAGGACTACGAGCATCATCGAG"),
                   id="my_seq_ID", name="made-up sequence",
                   description="Just some randomly typed ATGCs")
print(record)


[back to the toc](#toc)

<br>

### Writing to sequence files <a id='16'></a>
So far we have seen how to parse sequence files and work with sequence records. In many applications, we will also need to write our processed sequence records back into a standard sequence file. For this purpose, the `Bio.SeqIO` module has a `.write()` function. It can be thought as the reverse of the `.parse()` method we have learnt above.

In [ ]:
SeqIO.write(records, 'records.fasta', "fasta")

In [ ]:
! head -n 20 records.fasta


[back to the toc](#toc)

<br>

## `Seq` objects and 'Biological' methods <a id='10'></a>

Seq object are a flexible encapsulator for _sequence-like_ strings. 

In its essence, it behaves very much like a classical `str`, but it also has some
specific 'biological' methods, such has:

- complement()
- reverse_complement()
- transcribe()
- back_transcribe()
- translate()

These methods do not *require* any other extra arguments and they return a new `Seq` object.

#### Complementing and reverse complementing sequences

In [ ]:
intronless_dna = Seq(
    'CACCTCTGGAGCGGACTTATTTACCAAGCATTGGAGGAATATCGTAGGTAAAAATGCCTA'
    'TAGGATCCAAAGAGAGGCCAACATTTTTTGAAATTTTTAAGACACGCTGCAACAAAGCA')
print("intronless_dna:", intronless_dna)


# If we need to find its complement sequence:

compl_seq = intronless_dna.complement()
print()
print("compl_seq:", compl_seq)

# If we would need to find the reverse complement of our sequence, we can simply

rev_strand = intronless_dna.reverse_complement()
print()
print("rev_strand:", rev_strand)
print("alternatively:", compl_seq[::-1])

#### Transcription and reverse transcription of sequences

In [ ]:
# Let's now transcribe this piece of DNA into RNA. This is an intronless DNA sequence, so

rna_seq = intronless_dna.transcribe()
print("RNA:", rna_seq)

In [ ]:
# We can also reverse transcribe an RNA sequence to cDNA...
cdna_seq = rna_seq.back_transcribe()
cdna_seq

#### Translation of sequences

Up to here, we have described an intronless DNA sequence, and 'transcribed' it into an RNA sequence, `rna_seq`. If we know where the CDS starts, we can also translate the CDS into a protein sequence. In this example, the start codon can be found at 53. position.

In [ ]:
cds_seq = rna_seq[53:]
cds_seq

In [ ]:
protein_seq = cds_seq.translate()
protein_seq

The `.translate()` method can take extra arguments to finetune the translation specifics:

- **table** - Which codon table to use? Tables are based on [NCBI's Genetic Codes](https://www.ncbi.nlm.nih.gov/Taxonomy/Utils/wprintgc.cgi). Default is the "Standard" table.


- **stop_symbol** - Single character string, what to use for terminators.  Default is the asterisk, *


- **to_stop** - Boolean, defaults to False meaning do a full                                                        translation continuing on past any stop codons (translated as the                                                  specified stop_symbol).  If True, translation is terminated at                                                      the first in frame stop codon (and the stop_symbol is not                                                          appended to the returned protein sequence).


- **cds** - Boolean, indicates this is a complete CDS.  If True,                                                    this checks the sequence starts with a valid alternative start                                                      codon (which will be translated as methionine, M), that the                                                        sequence length is a multiple of three, and that there is a                                                        single in frame stop codon at the end (this will be excluded                                                        from the protein sequence, regardless of the to_stop option).                                                      If these tests fail, an exception is raised.


- **gap** - Single character string to denote symbol used for gaps. Defaults to the minus sign.

In [ ]:
# Codon tables can be defined by string names (NCBI genetic code table name)
# or integers (NCBI table ids)
codon_tables = [
    "Vertebrate Mitochondrial", # 2. The Vertebrate Mitochondrial Code
    6                           # 6. The Ciliate, Dasycladacean and Hexamita Nuclear Code
]

# to_stop is boolean, either True or False
to_stops = [True, False]

protein = cds_seq.translate(table="Vertebrate Mitochondrial",to_stop=True)
print(protein)
protein = cds_seq.translate(table="Vertebrate Mitochondrial",to_stop=False)
print(protein)
protein = cds_seq.translate(table=6,to_stop=True)
print(protein)
protein = cds_seq.translate(table=6,to_stop=False)
print(protein)

A potential source of error may come from the fact that two `Seq` objects may represent different types of biological sequences (protein and cDNA for instance).

Indeed, nothing prevents us from erroneously applying `Seq` methods to types of sequences that would 'biologically' not support these actions. For example, if we try to reverse-complement a protein sequence:

In [ ]:
protein_seq.reverse_complement() # returns a meaningless sequence by trying to interpret AA as IUPAC nts.

Or, we can concatenate two incompatible `Seq` objects together:

In [ ]:
print( cds_seq + protein_seq )


[back to the toc](#toc)

<br>

## Aligning sequences <a id='16'></a>
    
One may use any external software to perform multiple sequence alignment, 
but sometimes it may be more convenient to do it directly within tour code.


    

In [ ]:
from Bio import Align

aligner = Align.PairwiseAligner()

aln = aligner.align( "ATCGGCCGAT" , "TCGGACTAAGGCAGAT" )[0]
print(aln.score)
print(aln)

The default arguments of the aligner can be consulted and changed:

In [ ]:
print( "mode:" , aligner.mode )
print( "match_score:" , aligner.match_score )
print( "mismatch_score:" , aligner.mismatch_score )
print( "open_gap_score:" , aligner.open_gap_score )
print( "extend_gap_score:" , aligner.extend_gap_score )
print( "end_gap_score:" , aligner.end_gap_score )

In [ ]:
aligner.mode = 'global'
aligner.match_score = 2
aligner.open_gap_score = -1

aln = aligner.align( "ATCGGCCGAT" , "TCGGACTAAGGCAGAT" )[0]
print(aln.score)
print(aln)

Now for example, we could compute the alignment score of each of our records with another reference sequence.

In [ ]:
referenceSeq = 'MSNFQLSSVSFSSPILPSAVDDNSDTKLDVIRNTVNFSASIWGDQFLTFEEPDDLEMEKKVVEVLKEEVRKELVIKGSSNESLQHMKLIELIDVVQRLGVAYHFEEEIEEALKHIYVTYGEKWVDLNNLHNLSLWFRLLRQQGFNVSSGIFKYHMYEKGNFKESLCEDAQGLLALYEASYMRVEGEKVLDDALEFTKTHLAIIAKDPSCDSLLRAQIQEALKQPLRKRLPRLEAVRYIPIYQQDVSHSEVLLKLAKSDFNVLQSMHKEELSQICKWWKDLDMQKKLPYVRDRLIEGYFWILGIYFEPQHSHTRIFLMKTSMWLIVLDDTYDNYGTYEELKIFTDAVQRWSMSCLDLLPEYMKLIYQELLNHHQEMEESLEKEGKTYQIHYVIEMAKEVLENNLVEAKWLKEGYMPTLEEYMSVSMKTCTYGLMIARSFVGRVDNMVTEDTFKWVATYPPIVKAACLVLRLMDDITTHKEEQERGHVASSIECYQKETGASEKEACEFISNMVEDAWKVINRESLRPTDIPFSLLPSTINFARACDVIYKVNDSYTHARKEMINHIKSLLVHPLAI'

In [ ]:
%%time

alignments = {}

aligner = Align.PairwiseAligner()

for rec in  records :
    alignments[ rec.id ] = aligner.align( referenceSeq , rec.seq ).score
    
alignments


[back to the toc](#toc)

<br>

### Now, you can try to solve the exercises. <a id='18'></a>




<br>
<br>



[back to the toc](#toc)

<br>

# Additional Theory <a id='19'></a>
-----------------------------

The Biopython module is 'huge'! Under this section, we have included some more information on basic functionality which is not needed for the exercises.


[back to the toc](#toc)

<br>

### On `SeqRecord` iterators and processing large sequence files with `SeqIO` <a id='20'></a>

We have learnt how we can iterate over the sequence records contained in an input file using the `.parse()` method of `SeqIO`. There are some important limitations that come with iterators. Below, we will explore some of there limitations, how we can overcome them but at a cost in memory usage. Finally, some alternatives if we need to process large files without iterators.

In [ ]:
from Bio import SeqIO

In [ ]:
# A word about iterators. Once we iterate over all values an iterator gets empty.
# That means, we can not access the contents any longer.

# Let's create an iterator of sequence records:
records = SeqIO.parse("data/example.fa", "fasta")

# We can access the next item (in this case, the first one) via core next() method:
print("First record is", next(records).id)

# Now, let's iterate over all values with a for loop:
for record in records:
    print("first use", record.seq[:5])
print("We have reached the end of the iterator")

# At this point, we have reached the end of the iterator. We can not use it again:
for record in records:
    print("second use", record.seq[:5])

In [ ]:
# If we force the iterator with next(), it will not return a None, but a StopIteration exception
next(records)

Therefore, sometimes it is useful to convert the iterator into a list or a dictionary, given that it is not too big to fit into the memory. Iterators can be easily converted into lists, simply casting a `list()` function. For conversion into a dictionary, the `Bio.SeqIO` provides a specialized method called `.to_dict()`. 

In [ ]:
# Converting the sequence record iterator into a list

records = list(SeqIO.parse("data/example.fa", "fasta"))
# Now, let's loop over all values with for:
for record in records:
    print("first use", record.seq[:5])
# And again:
for record in records:
    print("second use", record.seq[:5])

In [ ]:
# Converting the sequence record iterator into a dictionary

records = SeqIO.parse("data/example.fa", "fasta")
rec_dict = SeqIO.to_dict(records)
print("Listing all items:")
for rec_id, rec in rec_dict.items():
    print(rec_id, rec.seq[:5])
print()

specific_rec_id = "ADY55933.1"
print("Accessing to a specific record:", specific_rec_id)
print(specific_rec_id, rec_dict[specific_rec_id].seq[:5])

For larger files, it isn’t possible to hold everything in memory, so `Bio.SeqIO.to_dict()` is not suitable. In these situations we can use the `Bio.SeqIO.index()` function. This will not populate a dictionary, rather index the input file such that we can access records in a an arbitrary order. This should be used with very large files; for small files it will be slower than the `.to_dict()` method.

In [ ]:
rec_dict = SeqIO.index("data/example.fa", "fasta")
print(specific_rec_id, rec_dict[specific_rec_id].seq[:5])


[back to the toc](#toc)

<br>

### More on `SeqFeature` objects <a id='21'></a>


`SeqFeature` object describes a feature along a sequence. Some of its useful attributes are:

- `.location` which tells us the 0-based coordinates of the feature
- `.type` which holds the type or the annotation of the feature
- `.qualifiers` is an ordered dictionary (the keys are always returned in same order) that holds qualifying information about the feature. These are analogous to the qualifiers from a GenBank feature table.

`SeqFeature` is defined within its own module: `Bio.SeqFeature`. For more detail on this object and the module, you can start with its [tutorial](http://biopython.org/DIST/docs/tutorial/Tutorial.html#htoc37) and [documentation](https://biopython.org/docs/1.74/api/Bio.SeqFeature.html).

Let's extract some useful information for our sequence record from its features.

In [ ]:
for feature in rec.features:
    print("From {} to {}, there is a '{}' feature".format(
        feature.location.start, feature.location.end, feature.type))
    for key, val in feature.qualifiers.items():
        print("    {} is {}".format(key, val))

When we slice a `SeqRecord` that contains `SeqFeature`s, we have learnt that only those features that completely fall within the slice will be kept. Then the positions will be offsetted relative to the new sliced sequence.

In [ ]:
# Let's grab the last record in our example file
records = list(SeqIO.parse("data/example.gp", "genbank"))
last_record = records[-1]

# We have seen above that the zinc binding site is from 47 to 178 (0-based)
zinc_binding = last_record[47:178]
incomplete_overlap = last_record[30:100]
print(zinc_binding)
print()

# Let's have a closer look at the single feature included within the sliced portion
feat = zinc_binding.features[0]
print("The feature is of '{}' type".format(feat.type))
print("It is from {} to {}".format(feat.location.start, feat.location.end))
print("Its location is {}".format(feat.location)) # We have the positions of 4 residues that make contact with Zn
print("Its qualifiers are: {}".format(feat.qualifiers))
print()

# On the other hand, the incompletely overlapping part (look at the features)
print(incomplete_overlap)